<p align="center">
  <img src="Graphics/Episode X.png" />
</p>

## (0) Sampling-Based Motion Planning

### (0.1) Reminder: The Motion Planning Problem

**Informal Definition:** Given a robot and a set of obstacles, plan a collision-free path from a
start position to a goal position.

**Formal Definition:** Let $R$ be a robot system with $d$ degrees of freedom, moving in a known environment cluttered with obstacles. Given start and target configurations $s$ and $t$ for $R$, decide whether there is a collision-free, continuous path $\gamma : [0, 1] \rightarrow \mathcal{X}_{free}$ such that $\gamma (0) = s$ and  $\gamma(1) = t$ and if so, plan such a motion.

### (0.2) Learning Outcomes

In this tutorial, we will cover:
* The fundamentals of sampling-based motion planning
* Some popular sampling-based motion planners: PRM, RRT, and RRT*

### (0.3) Disclaimer

This tutorial is heavily based on material from Oren Salzman's excellent Technion course ["Algorithmic Robot Motion Planning"](https://students.technion.ac.il/local/technionsearch/course/236901) (236901). It is taught every winter semester in the Faculty of Computer Science, and I highly recommend it for those of you who are interested in the topics we'll discuss today!
<p align="center">
  <img src="Graphics/MotionPlanner.gif" width=500/>
</p>

## (1) Sampling-Based Motion Planning

Today we will discuss one of my personal favorite class of algorithms, **sampling-based motion planners**. My affinity for them stems from their sheer simplicity and power, and hopefully you will enjoy them as I do after this tutorial!

### (1.1) Mo' Dimensions, Mo' Problems 

Up until now, we've tried to tackle simple versions of the motion planning problem using exact and grid-based methods such as:
- Trapezoidal Decomposition
- Visibility Graphs
- A* Grid Search

We focused on problems where we have a point robot translating among a set of polygonal (in the case of the first two) or non-polygonal (in the case of the third) obstacles through a 2D configuration space (C-space, $\mathcal{X}$), but in robotics we will often face situations where our dimensionality is larger than 2D and our environment is quite complex. In such cases, algorithms like the ones we've explored so far will simply fail to scale up efficiently to suit our needs. Another issue we face is the explicit representation of the free space $\mathcal{X}_{free}$ (and thus an explicit representation of the C-space) needed for these algorithms to work; while this is simple for a point robot (since it's identical to the workspace $\mathcal{W}$) and a bit less simple for a polygonal robot traversing convex polygonal obstalces (using a method involving [Minkowski sums](https://en.wikipedia.org/wiki/Minkowski_addition)), this will become more and more difficult as our environments become more complex.

Is all hope lost?

### (1.2) The Hero We Need

What if I told you there was a way to design algorithms whose complexity no longer (explicitly) depends on the dimensionality of the C-space and who do not require an explicit representation of $\mathcal{X}$, while still guaranteeing some form of completeness and optimality?
<p align="center">
  <img src="Graphics/Morpheus.jpg" width=500/>
</p>

Luckily for us, there is a way to rise above these challenges, and it relies on a little method called *sampling*. Using this method, we will approach the motion planning problem as follows:
1. Sample configurations ($d$-dimensional points) in the C-space, and check that these points do not collide with any obstacles in the workspace (using forward kinematics and geometric methods)
3. Use a nearest neighbors algorithm to connect points together as part of a roadmap (which can be a graph or a tree)
4. Use a local planner to check for edge collisions with obstacles (usually the most computationally expensive part)
5. Run a pathfinding algorithm (like A* or Dijkstra) on the final roadmap in order to find a path between the start and goal points

Does this method always guarantee a solution, and can it ever guarantee an optimal solution? Nope, at least not with the definitions for completeness and optimality that we've defined so far! We can describe two new concepts, though, that will give us a way to describe the guarantees offered by sampling-based motion planners:
- **Probabilistic Completeness**: An algorithm is probabilistically complete if, for a given motion planning problem, the probability of returning a solution tends to 1 as the number of samples $n$ tends to infinity.
- **Asymptotic Optimality**: An algorithm is asymptotically optimal if, for a given motion planning problem, the solution it returns converges to the optimal solution as the number of samples $n$ tends to infinity.

While we can no longer guarantee completeness or optimality in the classical sense for sampling-based planners, we can use the terms above to describe completeness and optimality guarantees for our algorithms in a *probabilistic* sense! We will use them to compare between various sampling-based algorithms soon, but first let's take a closer look at the process actually involved when we actually use these methods.

### (1.3) A Closer Look

#### (1.3.1) Sampling

As we saw in the last part, there are four stages in most sampling-based motion planning algorithms. Let's begin by examining the first stage of the whole process, which is the sampling of the configurations. Sampling methods can be broken up into two different approaches in general: deterministic methods and random methods. In this course, we will focus on random methods, which rely on some form of random number generators in order to choose which configurations to sample next from the C-space. The most common and simple random sampling method is uniform random sampling, whereby we will randomly sample a value for each DOF from a uniform distribution $U[a,b]$ (where $a,b$ are the lower and upper bounds of that DOF in the workspace). 

For example, say we have the following 2D workspace (left) and corresponding C-space (right):
<p align="center">
  <img src="Graphics/Sampling0.png" width=700/>
</p>

As we said before, we know nothing about the C-space (and we don't need to), and thus we represent it as being empty here (though of course it is not). We have sampled the purple point from $\mathcal{X}$, and now we want to check if this point is also in $\mathcal{X}_{free}$ so that we will be able to add it to our roadmap. Using forward kinematics, we can find the pose of the robot in our workspace in order to see if there are any collisions (we will discuss how we can actually detect them a bit later):
<p align="center">
  <img src="Graphics/Sampling1.png" width=700/>
</p>

We see that the robotic arm does indeed collide with the green obstacle in the workspace (left), and thus we will not add this point to our roadmap. We can also see the C-space representation for this collision on the right, though it is again important to note that we did not obtain this representation from this method (we would have to sample $n\rightarrow\infty$ points in order to build such a representation like that).

As part of our algorithm, we will not give up here, but rather we'll sample again from the C-space in the hopes of obtaining a point in the free space:
<p align="center">
  <img src="Graphics/Sampling2.png" width=700/>
</p>

Using forward kinematics again, we see that we've obtained a configuration for our robotic arm that sits in the free space, and thus can be added to our roadmap $\mathcal{G}$:
<p align="center">
  <img src="Graphics/Sampling3.png" width=700/>
</p>

Random sampling approaches display excellent practical performance and are very commonly used today, but sometimes we might want to have a bit more certainty and control over results for our motion planning problems (for example, NASA initially refused to use random sampling-based methods on their Mars rovers due to their desire to limit uncertainty wherever possible). In such cases, we could use determinstic sampling methods, which will return the same results every single time (this also has the advantage of allowing us to use offline computation to improve real-time execution). For example, we could use grids for this purpose (similar idea to what we saw last tutorial). We could also use sequences of numbers that appear random but are fully deterministic, such as Halton sequences:
<p align="center">
  <img src="Graphics/VanDerCorput.png" width=500/>
</p>
(This graphic actually shows the Van der Corput sequence, of which the Halton sequence is a generalization - either way, hopefully it gives you an idea of how we can obtain deterministic sequences of points that appear random.)

Finally, we might want to consider one problem with the lack of knowledge about obstacles when sampling (i.e. the lack of an explicit representation for $\mathcal{X}$): narrow passages. We had this sample problem last class, when we discussed grid-based search - sometimes we will run into cases where feasible paths must cross through some sort of narrow passage between obstacles, and if we don't sample enough points that we might not be able to reach the resolution needed for such paths. The figure below shows us an example of one such case:
<p align="center">
  <img src="Graphics/Narrow.png" width=400/>
</p>

Some alternative methods for dealing with such situations, which rely on some form of knowledge about the C-space, include medial-axis sampling (i.e. sample points far from obstacles) and obstacle-based sampling (i.e. sample points near obstacles). The former is exemplified below on the right, where it is compared to uniform sampling on the left:
<p align="center">
  <img src="Graphics/MedialAxis.png" width=600/>
</p>

and the latter is exemplified in the figure below (its potential utility for finding high-quality paths should remind you of visibility graphs):
<p align="center">
  <img src="Graphics/ObstacleBased.png" width=400/>
</p>

How such methods actually work is beyond the scope of this course, but it is always good to be aware of what exists out there!

#### (1.3.2) Nearest Neighbors

Once we take a sample and ensure that it is in the free space (i.e. we find $q_{rand}\in\mathcal{X}_{free}$), we add it as a vertex to the roadmap $\mathcal{G}$ and then seek to create edges between it and the other vertices already in $\mathcal{G}$. Rather than try to connect this sample to all vertices in $\mathcal{G}$, we'll prefer to only connect it to a certain subset of those vertices. We will choose those vertices to be the closest vertices to our sample, and we will use *nearest neighbor* (NN) algorithms to find them. There are two general approaches for deciding which/how many neighbors we want to consider:
- Connect the sample to all points within a distance of $r$ from it ($r$-NN)
- Connect the sample to the $k$ closest points to it ($k$-NN)

The metric for distance/closeness is determined by us (usually just Euclidean distance, which is easy to calculate), as is the value of $r$ or $k$. Once we've done so, conducting the nearest neighbors search is actually quite easy and can be done in $O(n)$ time (where $n$ is the number of vertices currently in $\mathcal{G}$). All we need to do is iterate over all of the vertices in $\mathcal{G}$, calculate the distances between them and our sample, and add them as neighbors if this distance is less than $r$ or keep a running list (heap) of the best $k$ neighbors.

We can make this search most efficient by storing the vertices of the roadmap in advanced data structures like Voronoi diagrams (shown below), randomly transformed grids (RTG), and $k$-d trees. 
<p align="center">
  <img src="Graphics/Voronoi.png" width=400/>
</p>

In Voronoi diagrams, for example, we have some set of points in space (we'll call them seeds), and for each seed we construct a cell in the diagram such that all points within that cell are closer to that seed than any other seed. Using such diagrams, conducting a nearest neighbors search is equivalent to identifying the Voronoi cell in which our sample is located, and returning the seed of that cell (i.e. this works for $k=1$, there are ways to modify it to work for $k>1$ but they're not so great).

Alas, I digress, and we shall move on to the next stage of sampling-based motion planning algorithms.

#### (1.3.3) Local Planning & Collision-Checking

When we add vertices (stage 1) or edges (stage 2) to our roadmap $\mathcal{G}$, we need to make sure that they do not incur any collisions with obstacle in the robot's workspace. This is a simpler task when it comes to vertices, since we only need to check that a single configuration is collision-free in the workspace. There are exact geometric methods for doing so, and this will usually be most efficient when we can model our obstacles and robot using polygons (checking intersections between line segments is relatively easy). In cases where the geometry of our robot or obstacles is too complex, we can use methods such as *bounding volumes*. In such cases, we will build a geometric object (for example, a rectangular prism) called a bounding volume that completely contains the object of interest, an example of which can be seen in the figure below:
<p align="center">
  <img src="Graphics/RabbitBV.png" width=400/>
</p>

Thus, if the robot (or its bounding volume) does not collide with any obstacles (or their bounding volumes) for a given configuration, then we know for certain that this configuration is indeed collision-free.

In the case of edges, things get a bit more complicated since we need to consider a continuous sequence of configurations and ensure that there are never any collisions. We can do this exactly using methods like computing *swept volumes*, but this is usually computationally expensive. An alternative is to just discretize the path (edge) into a set of configurations and check that those configurations are all collision-free (for example, using bounding volumes). For example:
<p align="center">
  <img src="Graphics/DiscretePath.png" width=500/>
</p>

Thus, we conduct collision-checking for all of the vertices and local planning for all of the edges in $\mathcal{G}$ to ensure that they are collision-free (otherwise we simply discard them), and we are left with a roadmap contained completely within the free space that can be traversed by our robot in a valid manner. But how exactly should our robot traverse $\mathcal{G}$ to get from the start to the goal?

#### (1.3.4) Pathfinding

Once we have our final roadmap $\mathcal{G}$ ready with collision-free vertices and edges, it's time to run a pathfinding algorithm like A* or Dijkstra on it in order to determine the best path between the start point and the goal point. It's important to note that most (if not all) sampling-based motion planning algorithms are classically incomplete, i.e. our roadmap might not contain any valid paths between the start and the goal. To increase the likelihood of returning a valid solution, we can always try increase the number of samples we take (if the algorithm is probabilistically complete, this is bound to work sooner or later) or increase the number of neighbors we consider (i.e. increase $k$ or $r).

Back to the search algorithms, you might remember that we discussed the A* algorithm in Tutorial #3, during which we saw that the A* search explores nodes in order of decreasing $f$ values, where $f(n)=g(n)+h(n)$ ($g(n)$ is the cost to go to the node $n$ from the start, and $h(n)$ is an admissible heuristic which underestimates the cost to come to the goal from that node). An example run of the A* algorithm on a graph is shown below:
<p align="center">
  <img src="Graphics/A_Star.gif" width=700/>
</p>

While we've mentioned Dijkstra's algorithm before (for example, Tutorial #9), we never formally discussed it. This algorithm conducts an uninformed search on a graph (not on trees) by order of lowest cost ($g$) value - in this respect it is similar to Uniform Cost Search (UCS). The general form of Dijkstra's algorithm does not terminate upon reaching the goal node (unlike UCS), but rather it will update the cost-to-go value (i.e. the $g$ value from the start node) for all nodes in the graph until they are all minimal. Thus, Dijkstra not only provides us with the lowest-cost path from the start node to the goal node, but also the lowest-cost path from the start node to any goal in the graph! We can modify it to terminate upon reaching a goal node if we only care about finding the lowest-cost path between the start and the goal. An example run of Dijkstra's algorithm on a graph (where $a$ is the start node and $b$ is the goal node) is shown below:
<p align="center">
  <img src="Graphics/Dijkstra.gif" width=600/>
</p>

Now that we (hopefully) understand the fundamentals of sampling-based motion planning, let's explore some of the most popular algorithms used in robotics today!

## (2) Probabilistic Roadmap (PRM)

The Probabilistic Roadmap algorithm was published by a team led by Lydia Kavraki (a very accomplished researcher in the field of robot motion planning) in 1996, and is conceptually one of the simplest sampling-based motion planners. It's also the first planner to ever demonstrate the ability to solve general planning problems in higher than 5 dimensions. It is a multi-query motion planner, meaning we can use the roadmap we build to solve motion planning problems multiple times for the same workspace (we just change the start and goal locations and re-add them to the roadmap). It follows the recipe we outlined above quite closely, and is both probabilistically complete and asymptotically optimal (sorry for the spoilers).

### (2.1) Algorithm

Let's explore the "simplified" version of this algorithm with the help of an example (there exists a "standard version", which is not asymptotically optimal, but we will not discuss it here). Assume we have a point robot translating in a 2D environment amongst a set of non-polygonal obstacles, and we want to find a path between a start point and a goal point:
<p align="center">
  <img src="Graphics/PRM0.png" width=400/>
</p>

We begin the algorithm by sampling a number of configurations in the robot's C-space, which is usually not explicitly represented. Such sampling might look as follows for our example:
<p align="center">
  <img src="Graphics/PRM1.png" width=400/>
</p>

Next, we run some collision-checking algorithm (in the workspace) on all of the samples and discard those which are not collision-free:
<p align="center">
  <img src="Graphics/PRM2.png" width=400/>
</p>

Now that we're sure we're only left with collision-free configurations, we can add them all to our roadmap. Then, we run a nearest neighbors algorithm (in our example, $r$-NN) on each new vertex in the roadmap in order to add potential edges to $\mathcal{G}$:
<p align="center">
  <img src="Graphics/PRM3.png" width=400/>
</p>

We must then check the validity of all of these new edges using a local planner in the workspace, and we'll discard any edges that aren't collision-free for their entirety:
<p align="center">
  <img src="Graphics/PRM4.png" width=400/>
</p>

Thus, we should be left with a roadmap $\mathcal{G}$ containing only collision-free vertices and edges:
<p align="center">
  <img src="Graphics/PRM5.png" width=400/>
</p>

Finally, we run a pathfinding algorithm (usually Dijkstra) on this roadmap in order to identify the lowest-cost (among all paths possible in the graph, this is not necessarily the optimal path in general though) path between the start and goal points, and return the result:
<p align="center">
  <img src="Graphics/PRM6.png" width=400/>
</p>
 
This algorithm is neither complete (why?) nor optimal (you can see this already in the figure above) in the classical sense, however it is both *probabilistically complete* and *asymptotically optimal* (for $r>0$ or $k>1$) ! If we sample $n\rightarrow\infty$ points in the C-space and add them to our roadmap, then we will have a probability of 1 to return a solution if one exists, and the solution we return will almost surely converge to the optimal one. What about complexity? Well, for $n$ samples if we use $r$-NN then our time complexity will be $O(n^{2})$, while if we use $k$-NN then our time complexity will be $O(n \cdot log(n))$ (there are more efficient $k$-NN algorithms available).

### (2.2) Code Implementation

A code implementation of the PRM algorithm can be found inside the `PRM` folder, and it requires the `scipy` library (just run `pip install scipy` in your terminal) for the $k$-NN part. To run this code, we can open up a terminal inside the TA#10 folder and run the following command:
```
python PRM/PRM_Main.py
```
By doing so, the script will randomly generate a new map with obstacles (dark orange), a start point (blue), and a goal point (green):
<p align="center">
  <img src="Graphics/PRM_Code0.png" width=600/>
</p>

Next, it will conduct the first few stages of the PRM algorithm (until the roadmap is fully built), and the resulting graph that would obtain might look as follows:
<p align="center">
  <img src="Graphics/PRM_Code1.png" width=600/>
</p>

Finally, a Dijkstra planner will search for a path across this roadmap and return it if one exists. Such a solution might look as follows (left - with PRM, right - no PRM):
<p align="center">
  <img src="Graphics/PRM_Code2.png" width=600/>
  <img src="Graphics/PRM_Code3.png" width=600/>
</p>

## (3) Rapidly Exploring Random Trees (RRT) and RRT*

The Rapidly Exploring Random Trees algorithm was published by a team led by Steven LaValle (another accomplished researcher in the field of robot motion planning) in 1998, and it's probably the most commonly used sampling-based motion planner today. RRT has proven to be an effective, conceptually simple algorithm for *single-query* planning in high-dimensional C-spaces. While it is probabilistically complete, it is *not* asymptotically optimal - however there is an asymptotically optimal version of the RRT algorithm called RRT*, which will learn today as well.

### (3.1) Algorithm

In RRT, we will do things a bit differently than we did with PRM. In this algorithm, we will build a tree whose root node is the start point, and which reaches a leaf node at the goal point - in this case, we will be able to recurse back to the root of the tree from the leaf, and so we won't need to perform any pathfinding on the roadmap we are obtaining! This makes things a bit easier.

Let's look at a tree which already has a few nodes and arcs, with the root node marked $x_{init}$:
<p align="center">
  <img src="Graphics/RRT0.png" width=600/>
</p> 

We will start by sampling nodes $x_{rand}$ one at a time, and for every sampled node we will connect them only to their nearest neighbor $x_{nearest}$ (i.e. we perform a $1$-NN search). This could look as follows:
<p align="center">
  <img src="Graphics/RRT1.png" width=567/>
  <img src="Graphics/RRT2.png" width=600/>
  <img src="Graphics/RRT3.png" width=570/>
</p> 

Using a local planner, we check that this proposed edge is collision-free in the workspace. If so, we'll add it to our tree as an arc:
<p align="center">
  <img src="Graphics/RRT4.png" width=600/>
</p> 

We repeat this process until the goal node is added to the tree with a corresponding arc, and once that occurs we perform our recursion step in order to find the path back to the root node (every node in the tree remembers who its parent node is). We are extremely unlikely to sample the exact goal node we want using methods like uniform sampling, and so we will define a parameter $p_{bias}$ called the *goal bias* which will describe the probability that we sample the goal node for any given sampling step. A common value used is 0.05, meaning that we will sample the goal node 5% of the time. The goal biasing method will encourage our tree to grow towards the goal, and the higher the goal bias value is the more our tree will want to grow towards the goal. Note that sampling the goal doesn't necessarily mean we will terminate in that step - if the straight-line path between the goal node and its nearest neighbor in the tree is not collision-free, then we must keep searching and growing our tree!

A common modification to the RRT algorithm that often improves its performance involves the use of a function called extend. Rather than directly connecting our sampled node to its nearest node in the tree, we will use the extend function to "move in the direction of" $x_{rand}$ from $x_{near}$ by a value of $\eta$ (this is called the step-size, and it is usually small). 
<p align="center">
  <img src="Graphics/RRTextend.png" width=600/>
</p> 
 
We will add the point $x_{new}$ that is $\eta$ distance away from $x_{near}$ in the direction of $x_{rand}$ to our tree then, instead of $x_{rand}$ itself. This will allow our tree to grow in a more "refined" and precise manner, and is the often preferred version of this algorithm. The following animation might help you get a better idea of the behavior of this algorithm:
<p align="center">
  <img src="Graphics/RRT.gif" width=600/>
</p> 

Rather than terminate immediately once the goal is added to the tree, we can alternatively set some total number of samples to take $n$, and continue sampling and building the tree until we've completed $n$ iterations. In such cases, we can improve the solution quality if new, better paths to the goal are found.


While this algorithm is *probabilistically complete*, it is unfortunately *not asymptotically optimal* - we will always return the first solution we obtain, regardless of its quality. Can we change the algorithm in any way to make it asymptotitcally optimal? Long story short, the answer is yes and it involves modified version of RRT called RRT*.

#### (3.3.1) RRT*

In order to make RRT asymptotically optimal, we can use a little trick called *rewiring* in order to further improve paths that we've just added to the tree. I will explain this trick visually using the following figures:
<p align="center">
  <img src="Graphics/RRT_Star0.png" width=550/>
  <img src="Graphics/RRT_Star1.png" width=600/>
  <img src="Graphics/RRT_Star2.png" width=600/>
  <img src="Graphics/RRT_Star3.png" width=550/>
  <img src="Graphics/RRT_Star4.png" width=600/>
  <img src="Graphics/RRT_Star5.png" width=600/>
  <img src="Graphics/RRT_Star6.png" width=550/>
  <img src="Graphics/RRT_Star7.png" width=600/>
  <img src="Graphics/RRT_Star8.png" width=600/>
</p> 

Note: The time complexity is $O(n \cdot log(n))$ for both RRT and RRT*!

Performance of RRT vs. RRT* over time:
<p align="center">
  <img src="Graphics/RRT_Graphs.png" width=1000/>
</p> 

### (3.2) Code Implementation

Let's now take a look at a code implementation of RRT and RRT* in Python. The relevant scripts and files can be found inside the `RRT` folder. First, we can import the necessary modules and libraries into this notebook:

In [ ]:
from RRT.RRTPlanner import RRTPlanner
from RRT.RRTStarPlanner import RRTStarPlanner
from RRT.MapEnvironment import MapEnvironment

Next, we'll prepare the environement for the problem. The C-space looks as follows, with the start point colored red, the goal point colored green, the obstacles colored yellow, and the free space colored purple:

In [ ]:
map = 'map1.json'
planning_env = MapEnvironment(json_file='RRT/'+map)
step_size = 0.2

Then, we can call the RRT planner and have it solve the problem:

In [ ]:
ext_mode = 'E2'
goal_prob = 0.05
RRT_planner = RRTPlanner(planning_env=planning_env, ext_mode=ext_mode, goal_prob=goal_prob)
RRT_plan = RRT_planner.plan()

We can visualize the final path according to the plan returned by RRT:

In [ ]:
RRT_planner.planning_env.visualize_map(plan=RRT_plan, tree_edges=RRT_planner.tree.get_edges_as_states());

We also call the RRT* planner and have it solve the problem:

In [ ]:
k = 5
RRTstar_planner = RRTStarPlanner(planning_env=planning_env, ext_mode=ext_mode, goal_prob=goal_prob, k=k)
RRTstar_plan = RRTstar_planner.plan()

Finally, we can visualize the final path according to the plan returned by RRT*:

In [ ]:
RRTstar_planner.planning_env.visualize_map(plan=RRTstar_plan, tree_edges=RRTstar_planner.tree.get_edges_as_states());

## (4) Conclusion

In this tutorial, we:
* Introduced the basic concepts of sampling-based motion planning algorithms
* Discussed popular sampling-based methods, such as PRM, RRT, and RRT*

Next week we will discuss the interface of task planning and motion planning, which is one of the most important fields in cognitive robotics today.

#### ***Credit:** This tutorial was written by Yotam Granov, Winter 2022.*

### **References**

[1] L. E. Kavraki, P. Svestka, J.C. Latombe and M. H. Overmars. ["Probabilistic roadmaps for path planning in high-dimensional configuration spaces"](https://ieeexplore.ieee.org/document/508439), 1996.

[2] [Motion Planning in Simple Geometric Spaces](http://motion.cs.illinois.edu/RoboticSystems/GeometricMotionPlanning.html)

[3] Oren Salzman's "Algorithmic Robot Motion Planning" Technion Course, 2022.

[4] S. A. Wilmarth, N. M. Amato and P. F. Stiller. ["MAPRM: A probabilistic roadmap planner with sampling on the medial axis of the free space"](https://ieeexplore.ieee.org/document/772448), 1999. 

[5] Sertac Karaman, Emilio Frazzoli. ["Sampling-Based Algorithms for Optimal Motion Planning"](https://arxiv.org/abs/1105.1186), 2011.

[6] Steven M. LaValle. ["Rapidly-exploring random trees: A new tool for path planning"](http://msl.cs.uiuc.edu/~lavalle/papers/Lav98c.pdf), 1998.

[7] Tim Chinenov, [Robotic Path Planning: PRM and PRM*](https://theclassytim.medium.com/robotic-path-planning-prm-prm-b4c64b1f5acb)'

[8] Tim Chinenov, [Robotic Path Planning: RRT and RRT*](https://theclassytim.medium.com/robotic-path-planning-rrt-and-rrt-212319121378)